In [1]:
import networkx as nx
import matplotlib.pylab as plt
%matplotlib inline
import os
import numpy as np
import pandas as pd

In [33]:
carpeta = os.getcwd()+'/'

G = nx.read_gml(carpeta+'dolphins.gml')

gender = pd.read_csv(carpeta+'dolphinsGender.txt', sep='\t', header=None)
delfines = gender[0] # nombres de delfines (nodos)
genero = gender[1]   # genero de los delfines en el mismo orden que los nodos

# A cada uno de los nodos ya existentes en el grafo, se le agrega el genero
for d,g in zip(delfines,genero):
    G.add_node(d, gender=g)


In [3]:
nombres = ['infomap','fast_greedy','edge_betweenness','louvain']

In [46]:
for metodo in nombres:
    data = pd.read_csv('./comparacion/labels_'+metodo+'.txt', sep='\t')
    dolphins = data['Nombre'] # nombres de delfines
    comu = data['Comunidad'] # comunidad a la que pertenecen
    
    for d,c in zip(dolphins,comu):
        G.node[d][metodo]=c

In [48]:
# Calculo de modularidad

delfines = G.nodes()
A = np.array(nx.to_numpy_matrix(G,nodelist=delfines)) # Creo la matriz de adyacencia
m = G.number_of_edges()
grado = dict(G.degree())

mod = np.zeros(4)

for j,metodo in enumerate(nombres):
    # Matriz de variable categorica comunidad: Cij=delta(ci,cj), tengo una por paso
    C = np.zeros(np.shape(A))
    for ni, idelfin in enumerate(delfines):
        for nj, jdelfin in enumerate(delfines):
            if G.node[idelfin][metodo] == G.node[jdelfin][metodo]:
                C[ni,nj]=1

    # Calculo de la modularidad
    # Q/Qmax = (S1-S2)*(1/2m)

    #S1 = Suma en nodos (Aij*Cij)
    #S2 = Suma en nodos(kikj*Cij/2m)
    S1 = 0
    S2 = 0

    for ni, idelfin in enumerate(delfines):
        for nj, jdelfin in enumerate(delfines):  
            S1 = S1 + A[ni,nj]*C[ni,nj]
            S2 = S2 + grado[idelfin]*grado[jdelfin]*C[ni,nj]
    S2 = S2/(2*m)
    mod[j] = (S1-S2)/(2*m)


In [51]:
df = pd.DataFrame({'Metodo':nombres, 'Modularidad':mod})
df

,Metodo,Modularidad
0,infomap,0.124461
1,fast_greedy,0.136684
2,edge_betweenness,0.132155
3,louvain,0.140244
